# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *H*

**Names:**

* *Antoine Basseto*
* *Andrea Pinto*
* *Jérémy Baffou*

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import numpy as np
import networkx as nx
import json
import epidemics_helper
import matplotlib.pyplot as plt
import pandas as pd
import part3_functions as part3

Before jumping into the exercices let's load our graph.

In [ ]:
with open("../data/nyc_augmented_network.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
nodes_list = list(map(lambda d : (d['id'], {'coordinates' : d['coordinates']}),data["nodes"]))
edges_list = list(map(lambda d : (d['source'],d['target']), data["links"]))

In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes_list)
G.add_edges_from(edges_list)

In [ ]:
part3.draw_graph(G)

---

## 2.3 Epidemics

#### Exercise 2.9: Simulate an epidemic outbreak

In [ ]:
SIR = epidemics_helper.SimulationSIR(G, beta=10.0, gamma=0.1)
SIR.launch_epidemic(source=23654, max_time=100.0)

In [ ]:
time_stamps_status, nodes_status = part3.nodes_status_over_time(SIR, 100, [1,3,30])

In [ ]:
part3.plot_population_status(nodes_status,percentage=True)

In [ ]:
part3.epidemic_markers(nodes_status,0.6)

In [ ]:
fig, axs = plt.subplots(len(time_stamps_status.values()), figsize=(400,500))
fig.suptitle('Epidemic evolution of the network')
i = 0
for stamps, status in time_stamps_status.items():
    axs[i].set_title("At day "+ str(stamps), fontsize=300)
    part3.draw_graph(G,nodes_status=status,ax=axs[i])
    i+=1

We can clearly see that the epidemy spread around the network really fast until it covers almost all of it.

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

In [ ]:
part3.strategy_1_simulation(nodes_list,edges_list, sim_nb=2, draw=False)

Not really efficient as the curves are really similar to the initial case.

In [ ]:
part3.strategy_1_simulation(nodes_list,edges_list,budget=10000)

More effective but still not really good as we cut around 20% of the network edges

#### Exercise 2.11: Strategy 2

Strategies possibles : 
- enlenver les nodes avec un haut degree pour eviter propagation exponentielle (risque de dépenser beaucoup du budget)
- enlever les ponts du graph pour renforcer le nombre d ilots (risque de dépenser beaucoup dans le cut de node seul)
- regarder les communautées et essayer de les séparées les unes des autres.
- calculer la edge_betweenness centrality et cut ceux qui ont la plus grande.

pistes :

- https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html#networkx.algorithms.centrality.edge_betweenness_centrality
- https://networkx.org/documentation/stable/reference/algorithms/community.html
- https://networkx.org/documentation/stable/reference/algorithms/clustering.html
- https://networkx.org/documentation/stable/reference/algorithms/centrality.html
- https://networkx.org/documentation/stable/reference/algorithms/bridges.html

### High betweenness method

In [ ]:
max_centered_edges = part3.extract_max_centered_edges() #have been computed using the cluster and the function https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html#networkx.algorithms.centrality.edge_betweenness_centrality

In [ ]:
Graph_high_centrality = nx.nx.Graph()
Graph_high_centrality.add_nodes_from(nodes_list)
Graph_high_centrality.add_edges_from(max_centered_edges[:2500])
part3.draw_graph(Graph_high_centrality,title="View of high centrality edges",edge_width=10.0)

In [ ]:
part3.strategy_2_simulation(nodes_list, edges_list, max_centered_edges, sim_nb=2)

### Vaccination method (high degree node)

In [ ]:
part3.strategy_3_simulation(nodes_list, edges_list, budget=1000)

### Community method

In [ ]:
communities = nx.algorithms.community.greedy_modularity_communities(G)

In [ ]:
part3.draw_communities(G, communities, boundaries=True)

In [ ]:
part3.strategy_4_simulation(nodes_list,edges_list)

## Cost/Effectiveness comparison

In [ ]:
random = []
vaccination = []
betweenness = []
community = []
strategies = ["random","vaccination","betweenness","community"]
budgets = [100,200,300,400,500,600,700,800,900,1000,1500,2000,2500,5000,7500,10000]
for b in budgets:
    random.append(compute_mean_susceptible(nodes_list, edges_list, b, "random"))
    vaccination.append(compute_mean_susceptible(nodes_list, edges_list, b, "vaccination"))
    betweenness.append(compute_mean_susceptible(nodes_list, edges_list, b, "betweenness"))
    community.append(compute_mean_susceptible(nodes_list, edges_list, b, "community"))
fig, ax = plt.subplots()
ax.set_title("Comparison of ") 